In [1]:
import plotly.express as px
import pandas as pd
import pymssql
import json

In [40]:
database = "arctic_analysts_capstone"
user = "arctic_analysts"
password  = "ThisPassw0rd!"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

def get_data(n = False):
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor()
    
    query = f"SELECT DISTINCT Date FROM building_permits"

    available_years_df = pd.read_sql(query, conn)
    available.sort_values(by = 'Date', inplace = True)
    available_dates = available.Date.astype('str').tolist()
    
    picked_date = available_dates[n]
    columns_to_select = 'CountyFips, County, StateFips, [1_Unit]'
    query = f"SELECT {columns_to_select} FROM building_permits WHERE building_permits.Date = '{picked_date}'"
    bpm_df = pd.read_sql(query, conn)
    print(f'\nBefore Data Management: {available_years_df.shape[0]}')

    return bpm_df

In [42]:
n = 0
while n < 300:
    bpm_df = get_data(n)
    print(bpm_df['1_Unit'].sum())
    n += 1


Before Data Management: 267
54170

Before Data Management: 267
59577

Before Data Management: 267
78625

Before Data Management: 267
68931

Before Data Management: 267
77372

Before Data Management: 267
76578

Before Data Management: 267
64283

Before Data Management: 267
73218

Before Data Management: 267
63234

Before Data Management: 267
67429

Before Data Management: 267
59560

Before Data Management: 267
54159

Before Data Management: 267
61779

Before Data Management: 267
60986

Before Data Management: 267
78003

Before Data Management: 267
79041

Before Data Management: 267
83612

Before Data Management: 267
80645

Before Data Management: 267
74902

Before Data Management: 267
78419

Before Data Management: 267
62831

Before Data Management: 267
69967

Before Data Management: 267
58896

Before Data Management: 267
55997


KeyboardInterrupt: 

In [4]:
bpm_df = bpm_df[["CountyFips", "County", "StateFips", "1_Unit"]].copy()

In [5]:
bpm_df

,CountyFips,County,StateFips,1_Unit
0,001,Autauga County ...,01,33
1,003,Baldwin County ...,01,275
2,005,Barbour County ...,01,2
3,007,Bibb County ...,01,0
4,009,Blount County ...,01,1
...,...,...,...,...
3020,037,Sweetwater County ...,56,2
3021,039,Teton County ...,56,16
3022,041,Uinta County ...,56,1
3023,043,Washakie County ...,56,0


In [21]:
bpm_df['locale_fips'] = bpm_df.StateFips + bpm_df.CountyFips
bpm_df

,CountyFips,County,StateFips,1_Unit,locale_fips
0,001,Autauga County ...,01,33,01001
1,003,Baldwin County ...,01,275,01003
2,005,Barbour County ...,01,2,01005
3,007,Bibb County ...,01,0,01007
4,009,Blount County ...,01,1,01009
...,...,...,...,...,...
3020,037,Sweetwater County ...,56,2,56037
3021,039,Teton County ...,56,16,56039
3022,041,Uinta County ...,56,1,56041
3023,043,Washakie County ...,56,0,56043


In [23]:
with open('2021_US_Counties_3.7.json') as file:
    counties = json.load(file)

In [25]:
fig = px.choropleth(bpm_df, geojson=counties, locations = 'locale_fips', color = '1_Unit',
                   color_continuous_scale = "rdylgn_r", scope = "usa")

In [27]:
with open("test_map.html", 'w') as mapfile:
    mapfile.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
mapfile.close()